# NDex Client Tutorial

The NDEx Python Client is a module that simplifies access to the NDEx Server API and provides convenience methods 
for common operations on networks. Detailed technical info about the Python Client is available in the relevant 
section of our NDEx v1.3 Technical Design Document.


In this tutorial, we will first show you how to install the NDEx Python Client module and use it for basic operations on networks stored on an NDEx server. The NDEx Python Client 2.0 requires Python 2.7.9 and the latest version of the PIP Python package manager for installation.



The Python Client is available as a development release of the ndex module in the Python Package Index (PyPI) as of 
June 2015.  This tutorial was composed using the 2.0 release of the ndex module. To install this module to your computer using PIP:
        >>> pip install ndex
or
        >>> pip install ndex
        
if you have an older version of this module already installed. 


## Setting up an NDEx Client

With the ndex module installed, start Python and import ndex.client and the io module for python:

In [1]:
import ndex.client as nc
import io

In this section you will configure two client objects to access the public NDEx server. The first (shown as "anon_ndex" below) will let you make anonymous requests and the second (shown as "my_ndex")will let you perform operations requiring authentication, such as saving networks to your account.

In [2]:

anon_ndex=nc.Ndex("http://public.ndexbio.org")

#For convenience and clarity in this example, we set the variables ‘my_account’ and ‘my_password’ to hold the strings
#for your account name and password:                 
my_account="(your username here)"
my_password="(your password here)"
my_ndex=nc.Ndex("http://public.ndexbio.org",my_account,my_password)


## Working with NDEx Network Using the Client

### Get Network Information by Accession

You can access a network by its accession id, which is an universally unique identifier(UUID) assigned to the network by the NDEx server. All networks have a UUID and they are unique across all servers. No two networks will share an UUID.

In this step, you will get basic information about the network and retrieve a Network Summary structure. The ‘Metabolism of RNA’ network is in the NDEx Tutorials account on the public NDEx server; its UUID is ‘9ed0cd55-9ac0-11e4-9499-000c29202374’


anon_ndex will access the network using the method 'get_network_summary(network_id)' which will return a list of 
Network Summary dictionaries. 

In [3]:

ns=anon_ndex.get_network_summary('9ed0cd55-9ac0-11e4-9499-000c29202374')
print ns.get('name')
print ns.get('edgeCount')
print ns.get('nodeCount')

Metabolism of RNA
4344
361


### Find Networks by Search – Simple Search


You can search for networks by the text in their name and description as well as the names and controlled vocabulary terms associated with their nodes. The input is a search string that conforms to Lucene search string syntax, but in its simplest form is one or more search terms separated by spaces.


In [4]:

metabolic_networks = anon_ndex.search_networks('metabo*','ndextutorials', block_size=2)
print len(metabolic_networks)
for ns in metabolic_networks: print ns.get('name')

2
Metabolism
Metabolism of proteins


### Get a Network


You can obtain an entire network as a CX stream, which is an NDEx format that is optimized for streaming networks. 
This is performed as a monolithic operation, so care should be taken when requesting very large networks. Applications
can use the get_network_summary method to check the node and edge counts for a network before attempting to use 
get_complete_network. The stream is contained in a Response object from the Python requests library 
(http://docs.python-requests.org/en/master/).


anon_ndex will use the method 'get_complete_network(network_id)' to get a Response object (from Python requests library)

In [5]:

response =  anon_ndex.get_network_as_cx_stream('9ed0cd55-9ac0-11e4-9499-000c29202374')
print len(response.json())

6166


### Query a Network – Neighborhood Query



You can retrieve a ‘neighborhood’ subnetwork of a network as CX stream. The query finds the subnetwork by first 
identifying nodes that are associated with identifiers in the search_string, then traversing search_depth number of 
steps from those nodes.  The search_depth is limited to 3 steps and the following example shows a query using the 
default 1-step limit. Once against, the stream is contained in a Response object from the Python requests library 
(http://docs.python-requests.org/en/master/).


anon_ndex will use the method 'get_neighborhood_as_cx (network_id, search_string, search_depth=1)' 
to get a Response object (from Python requests library)

In [6]:

response= anon_ndex.get_neighborhood_as_cx_stream('9ed0cd55-9ac0-11e4-9499-000c29202374', 'XRN1')
print len(response.json())

50


## Create a Network


You can create a new network on an NDEx server if you have a CX stream. The network is created in the user account 
associated with the client object. All methods that create or modify content on the NDEx server require authentication,
so you will use the my_ndex client object that you set up at the start of the tutorial and to create a network in 
your account.
An error is returned if the CX stream is larger than a maximum size for network creation set in the NDEx server 
configuration.


First convert the Response object from a bytes-like objects into bytes objects.Then use 
'save_cx_stream_as_new_network(network)' that will return the ID of the network as an integer.

The small network that you obtained based on a query for 10 edges (in the previous section) will be saved to your 
account in this example. A Network Summary dictonary for the new network is returned so that you can obtain the 
UUID assigned to the network.

In [7]:

cx_stream = io.BytesIO(response.content)
new_network_id = my_ndex.save_cx_stream_as_new_network(cx_stream)
print new_network_id

2e4bc25c-3e3e-11e6-a5c7-06603eb7f303


## Updating

### Update a Network

You can also update (overwrite, replace) a network on an NDEx server based on a network CX stream. The updated network
must be owned by the user account associated with the client object. Use the method 
"update_cx_network(self, cx_stream, network_id, provenance=None)" which will return the ID of network modified 
(same as network_id)

### Update Network Profile

You can also update just the network profile (defined as the name, description, version, and visibility of a network) 
on an NDEx server based on the network UUID. You use the method "update_network_profile(network_id, network_profile)" 
which does not return anything.

In [8]:

network_profile = {"name":"Renamed Network","description":"New Description","version":"2.0","visibility":"PUBLIC"}
my_ndex.update_network_profile(new_network_id,network_profile)

''

### Set Read-Only

You can also set a network to be read only using "set_read_only(network_id, boolean)" so that it can be retrieved more
quickly by others. The method does not return anything.Please note that a read only network cannot be edited or 
deleted, so you will need to revert it to its original state  prior to proceeding with the next step in this tutorial. 

In [9]:


my_ndex.set_read_only(new_network_id,True)

#To revert to its original state:

my_ndex.set_read_only(new_network_id,False)

1467143408395L

### Delete a Network

Finally, you can delete a network using "delete_network(networkId)" that you own by its UUID. The method does not 
return anything. Be careful: there is no method to undo a deletion.

In [10]:

my_ndex.delete_network(new_network_id)

## Search for Networks By Network Properties

You can get a list of networks who have certain properties with a given value. For example, the following query finds all networks on public with a format of Cytoscape-XGMML.

In [11]:
query = {"properties":[{"name":"ORGANISM","value":"http://identifiers.org/taxonomy/9606"}]}
networks = my_ndex.search_networks_by_property_filter(query,account_name='ndextutorials')
for network in networks:
    print "Network: " + network.get("name")
    print "Properties: "
    for prop in network.get("properties"):
        print "  " + prop.get("predicateString") + " : " + prop.get("value")

Network: Metabolism of RNA
Properties: 
  ORGANISM : http://identifiers.org/taxonomy/9606
  URI : http://purl.org/pc2/4/Pathway_098d5b97ab0d39bfcf905771f06d18a5
  Source : http://purl.org/pc2/4/reactome46_human
  sourceFormat : SIF
